In [42]:
#NOTE IF WE CANT INSTALL PYSPARK IN THE CONDA ENVIRONMENT USING CONDA INSTALL 
#USR !pip3 install in code and it will work and go the the specific environment
#!pip3 install pyspark


In [43]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, FloatType
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import functions as f
from pyspark.sql import types as t
import psycopg2
from sqlalchemy import create_engine
from pathlib import Path

spark = SparkSession.builder.appName('final').getOrCreate()

#The columns are in capital and small so we have to make it case, sensetive. By default its off.
spark.conf.set('spark.sql.caseSensitive', True)

file_path = '/Users/pratyushpradhan/Developer/Personal/Projects/Crypto/RealTimeData'
df = spark.read.option('header','true').csv(file_path, inferSchema=True)


In [44]:

#TODO:  6 perform analytics task thorugh filter and search group by columns

#TRANSFORMATION

#drop columns
df= df.drop('B','x','f','L')

#rename
col_list=['start_time','close_time','symbol','interval','open','close','high','low','base_volume','num_trades'
,'quote_volume','taker_buy_base_volume','taker_buy_quote_volume']
df = df.toDF(*col_list)

#drop column if value null
df= df.na.drop(how='any', subset=['symbol','start_time','close_time','open','close','high','low','base_volume'])

In [45]:
#update date values from unix epoch timestamp to timestamp
#NOTE Unix epoch time is utc by default
df = df.withColumn('start_time', f.to_timestamp(df.start_time/1000)).withColumn('close_time', f.to_timestamp(df.close_time/1000))

In [46]:
#get distinct symbols from all columns
dist_symbols = df.select('symbol').distinct().rdd.flatMap(lambda x:x).collect()


In [47]:
def createConnection(database,user,password,host,port):
    return create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}?client_encoding=utf8")

#load data to database
#TODO: CREATE CONFIG fiLE
database='crypto'
user='cryptouser'
password= 'secretcrypto' 
host='127.0.0.1'
port= '5432'

#TODO: implement try catch  and logging
engine = createConnection(database,user,password,host,port)

In [54]:
temp = df.filter(f"symbol = 'ethusdt' ").toPandas()
display(temp)

,start_time,close_time,symbol,interval,open,close,high,low,base_volume,num_trades,quote_volume,taker_buy_base_volume,taker_buy_quote_volume


In [49]:
for value in dist_symbols:
    #note that the data table name is same as symbol name
    # TODO: if time add a query so that if new symbol comes and its corresponding datatable does not exist, create new table with a predefined schema
    #NOTE: always  " " inside filter at start function no ''
    #NOTE: for postgres take db in lowercase, we need to add  quotes (" ") always if taken to uppercase
    df.filter(f"symbol = '{value}' ").toPandas().to_sql(value.lower(), engine, index=False, if_exists= 'append')

AnalysisException: Column 'btcusdt' does not exist. Did you mean one of the following? [close, high, interval, low, open, symbol, start_time, close_time, num_trades, base_volume, quote_volume, taker_buy_base_volume, taker_buy_quote_volume]; line 1 pos 9;
'Filter (symbol#1091 = 'btcusdt)
+- Project [start_time#1128, to_timestamp((cast(close_time#1090L as double) / cast(1000 as double)), None, TimestampType, Some(America/Chicago)) AS close_time#1142, symbol#1091, interval#1092, open#1093, close#1094, high#1095, low#1096, base_volume#1097, num_trades#1098, quote_volume#1099, taker_buy_base_volume#1100, taker_buy_quote_volume#1101]
   +- Project [to_timestamp((cast(start_time#1089L as double) / cast(1000 as double)), None, TimestampType, Some(America/Chicago)) AS start_time#1128, close_time#1090L, symbol#1091, interval#1092, open#1093, close#1094, high#1095, low#1096, base_volume#1097, num_trades#1098, quote_volume#1099, taker_buy_base_volume#1100, taker_buy_quote_volume#1101]
      +- Fil

In [ ]:
# temp = df.filter("symbol = 'ETHUSDT' ").toPandas().to_sql("ETHUSDT", engine, index=False, if_exists= 'append')
# display(temp)
#.to_sql('ethusdt', engine, index=False, if_exists= 'append')

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


30

In [ ]:
#TODO: IMPLEMENT AT LAST
#code to delete all the csv files in the folder after all the operation is complete

# for path in Path(file_path).glob('*'):
#     if path.is_file():
#         path.unlink()

    "t": 123400000, // Kline start time
    "T": 123460000, // Kline close time
    "s": "BNBBTC",  // Symbol
    "i": "1m",      // Interval
    "o": "0.0010",  // Open price
    "c": "0.0020",  // Close price
    "h": "0.0025",  // High price
    "l": "0.0015",  // Low price
    "v": "1000",    //     "t": 123400000, // Kline start time
    "T": 123460000, // Kline close time
    "s": "BNBBTC",  // Symbol
    "i": "1m",      // Interval
    "o": "0.0010",  // Open price
    "c": "0.0020",  // Close price
    "h": "0.0025",  // High price
    "l": "0.0015",  // Low price
    "v": "1000",    // Base asset volume : the volume of the crypto you are buying
    "n": 100,       // Number of trades
    "q": "1.0000",  // Quote asset volume : the volume of the crypto you are selling
    "V": "500",     // Taker buy base asset volume : explained below // The way i understand it its the volume where the buyer was the taker
    "Q": "0.500",   // Taker buy quote asset volume

Best explanation: 

- BNBBTC indicates BNB bought by using BTC
like HDLRUPEE

- BTC/USDT 8,243.35 (at the time of writing 10/2/19, 11:10 UTC, source HitBTC), means the base currency 1 BTC is traded against the quote currency USDT and exchanged for 8,243.35 USDT.

- In a given market FOO/BAR, baseVolume indicates the total amount of FOO that has been bought and sold in this market in the past 24hrs, while quoteVolume is the total amount of BAR that has been bought and sold in this market in the past 24hrs. 
One more note: when you buy FOO on the FOO/BAR market, you are buying it for BAR, right? So, when you buy FOO for BAR, that means, you are actually selling BAR for FOO.

- Buyer as a taker/Maker explained: For a trade to happen, there must be a buyer and a seller. The buyer could have placed an offer to buy, which the seller took. Or the seller could have placed an offer to sell, which the buyer took. These are the only two possibilities.
If the buyer placed the offer which the seller later took, the buyer is the maker (he made liquidity available) and the seller is the taker (they took the buyer's offer). If the seller placed the offer which the buyer later took, the seller is the maker (he made liquidity available) and the buyer is the taker (they took the seller's offer).

This matters for at least two reasons:

1) Typically the maker pays a lower fee than the taker.

2) This makes a difference in understanding what the price is telling you. For example, imagine if there's a market with people willing to sell apples for $1 and willing to buy apples for $0.90 -- if the price never changes, you will see transactions for $1 and transactions for $0.90, which might make you think the price is changing. But all the $1 trades will have the "buyer is not maker" flag and all the $0.90 trades will have the "buyer is maker" flag, allowing you to understand that the change in trade prices doesn't reflect any actual change in the market.

- TAKER BUY Base asset volume:
    Taker buy base asset volume represents how many of the total base asset volume are contributed by the taker buy orders. Once    taker_buy_base_asset_volume and the total volume (Volume) is known, the value of the following cases ( taker_sell_volume, maker_buy_volume,    maker_sell_volume ) are clear. 

    Every trade has a buyer and a seller. A buyer can be a maker or a taker. But when a buyer is a maker, the seller must be a taker, and vice versa. That is,

    taker_buy_base_asset_volume = maker_sell_base_asset_volume

    taker_sell_base_asset_volume = maker_buy_base_asset_volume

    total_volume = taker_buy_base_asset_volume + taker_sell_base_asset_volume
    = maker_buy_base_asset_volume + maker_sell_base_asset_volume



Quote asset volume is total volume in units of the quote currency. 
Taker asset volume is the volume of market orders in units of the base and quote currency.
    "n": 100,       // Number of trades
    "q": "1.0000",  // Quote asset volume
    "V": "500",     // Taker buy base asset volume
    "Q": "0.500",   // Taker buy quote asset volume

Quote asset volume is total volume in units of the quote currency. 
Taker asset volume is the volume of market orders in units of the base and quote currency.

